In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize


accepts = pd.read_csv('../Data/chefmozaccepts.csv')
cuisine = pd.read_csv('../Data/chefmozcuisine.csv')
hours = pd.read_csv('../Data/chefmozhours4.csv')
parking = pd.read_csv('../Data/chefmozparking.csv')
geo = pd.read_csv('../Data/geoplaces2.csv') 
usercuisine = pd.read_csv('../Data/usercuisine.csv')
payment = pd.read_csv('../Data/userpayment.csv')
profile = pd.read_csv('../Data/userprofile.csv')
rating = pd.read_csv('../Data/rating_final.csv')


In [11]:
overall_rating = pd.DataFrame( np.zeros((len(res_all),len(user_all)))-1.0, 
                              columns=user_all, index=res_all )
food_rating = overall_rating.copy()
service_rating = overall_rating.copy() 

for r, u, o, f, s in zip(rating.placeID, rating.userID, rating.rating, rating.food_rating, 
                         rating.service_rating):
    overall_rating.loc[r,u] = o
    food_rating.loc[r,u] = f
    service_rating.loc[r,u] = s
#User Pair has rating
review = pd.DataFrame( np.zeros(overall_rating.shape), columns=user_all, index=res_all)
review[overall_rating >= 0] = 1
res_cuisine = pd.get_dummies(cuisine,columns=['Rcuisine'])

# remove duplicate restaurant ID's. 
# A restaurant with multiple cuisine categories would have multiple columns equal 1
res_cuisine = res_cuisine.groupby('placeID',as_index=False).sum()
res_parking = parking.copy()
res_parking.parking_lot = res_parking.parking_lot.map({'fee':1, 'none':0, 'public':1, 'yes':2,
                                        'street':1, 'valet parking':1, 'validated parking':1})

# remove duplicate restaurant ID's. 
# A restaurant with multiple parking options may have a value > 2
res_parking = res_parking.groupby('placeID',as_index=False).sum()

res_info = geo[['latitude','longitude','placeID','name','address','city','state']]

# These features should be relevant for rating prediction since they are about services 
# and price. Especially, 'alcohol','smoking_area', and 'price' relate to 'drink_level', 
# 'smoker', and 'budget' in the user profiles 
res_service_price = geo[['placeID','alcohol','smoking_area','other_services','price']]

# 1 if alcohol is available, 0 otherwise
res_service_price.alcohol = res_service_price.alcohol.map(lambda x: 0 if x == 'No_Alcohol_Served' else 1)
# 1 if there is smoking area, 0 otherwise
res_service_price.smoking_area = res_service_price.smoking_area.map(lambda x: 0 if (x == 'none') | (x == 'not permitted') else 1)
# 1 if other services are available, 0 otherwise
res_service_price.other_services = res_service_price.other_services.map(lambda x: 0 if x == 'none'  else 1)
# map price levels to numbers
res_service_price.price = res_service_price.price.map({'low': 1, 'medium': 2, 'high': 3})


NameError: name 'res_all' is not defined

In [8]:
df_res = pd.DataFrame({'placeID': res_all})
df_res = pd.merge(left=df_res, right=res_cuisine, how="left", on="placeID")
df_res = pd.merge(left=df_res, right=res_parking, how="left", on="placeID")
df_res = pd.merge(left=df_res, right=res_service_price, how="left", on="placeID")
df_res = pd.merge(left=df_res, right=res_environment, how="left", on="placeID")

# print(df_res.shape)
df_res.head()

                         address  \
0           30 Eglinton Avenue W   
1      10110 Johnston Rd, Ste 15   
2        2450 E Indian School Rd   
3                5981 Andrews Rd   
4   1775 E Tropicana Ave, Ste 29   
5          6055 E Lake Mead Blvd   
6             6401 Morrison Blvd   
7              2825 32 Avenue NE   
8                  436 Market St   
9           825 Commonwealth Ave   
10    13843 N Tatum Blvd, Ste 15   
11            529-17th Avenue SW   
12       619 Mount Pleasant Road   
13             10897 Rue du Pont   
14           1614 Queen Street E   

                                           attributes             business_id  \
0   {'RestaurantsReservations': 'True', 'GoodForMe...  QXAEGFB4oINsVuTFxEYKFQ   
1   {'GoodForKids': 'True', 'NoiseLevel': 'u'avera...  gnKjwL_1w79qoiV3IC_xQQ   
2   {'RestaurantsTakeOut': 'True', 'BusinessParkin...  1Dfx3zM-rW4n-31KeC8sJg   
3   {'RestaurantsPriceRange2': '2', 'BusinessAccep...  fweCYi8FmbJXHCqLnwuk8w   
4   {'OutdoorSeati

In [9]:
relevant_business.to_csv("temp.csv",index=False)